In [1]:
# 필요한 모듈 임포트
!pip install snscrape
import snscrape.modules.twitter as sntwitter
import pandas as pd

!pip install konlpy
from konlpy.tag import Okt
okt = Okt()

import re

In [2]:
# Query by text search
# 시작일부터 끝 날까지 데이터를 반환하는 함수

def get_df(since, until):
    maxTweets = 5000

    # Creating list to append tweet data to
    tweets_list = []


    # Using TwitterSearchScraper to scrape data and append tweets to list
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(f'느린마을 막걸리 since:{since} until:{until}').get_items()):
        if i>maxTweets:
            break
        tweets_list.append([tweet.date, tweet.id, tweet.content, tweet.user.username])

    # Creating a dataframe from the tweets list above
    tweets_df = pd.DataFrame(tweets_list, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])
    
    return tweets_df

In [3]:
tweets_2019 = get_df('2019-01-01','2019-12-31')
tweets_2020 = get_df('2020-01-01','2020-12-31')
tweets_2021 = get_df('2021-01-01','2021-12-31')

In [4]:
# 각 속성별 빈도수 데이터 : 리스트로 가져오기

# 속성별로 분류한 텍스트 데이터에서 딕셔너리 가져오는 함수 
def get_dict(file):
    with open(file,'r',encoding='UTF8') as f:
        sents = []
        while True:
            line = f.readline()
            if not line: # '' 의 길이는 1, 컨텐츠가  없을 때는 0
                break
            sents.append(line.strip().replace(",","").replace("'","").replace("(","").replace(")","").split())
        
        fdist = dict()
        
        for sent in sents:
            fdist[sent[0]] = int(sent[1])
        
        sorted(fdist.items(), key=lambda x: x[1]) # 빈도수 높은 순서대로 내림차순 정렬
        
        return fdist

#데이터 리스트로 저장
health = list(get_dict('./data/health.txt').keys())
quality_stability = list(get_dict('./data/quality&stability.txt').keys())
price = list(get_dict('./data/price.txt').keys())
flavor = list(get_dict('./data/flavor.txt').keys())
brand = list(get_dict('./data/brand.txt').keys())

In [5]:
properties = []
properties.append(health)
properties.append(quality_stability)
properties.append(price)
properties.append(flavor)
properties.append(brand)

In [10]:
#특수문자 불러오기
import string
punct = list(string.punctuation)

In [11]:
# 불용어 사전 불러오기
stop_words = []

with open("stopWords.txt", "r",encoding='UTF8') as f:
    data = f.read()

stop_words = data.split('\n')

with open("my_stopWords.txt", "r",encoding='UTF8') as f:
    data2 = f.read()

stop_words.extend(data2.split('\n'))

# 기본 불용어 사전에 불용어 추가하기
user_stop_words = ['amp','旅がえり','gt','느린','마을','막걸리']
stop_words.extend(user_stop_words)

In [12]:
# 전처리 함수
# 문자열 문장을 넣으면 문자열에 대해 전처리한 토큰 리스트를 반환하는 함수

def process_sents(sent):
    sent = re.compile(r'https?://[a-zA-Z0-9_/:%#\$&\?\(\)~\.=+-]*').sub('', sent) # url -> 삭제
    sent = re.compile(r'(@)[a-zA-Z0-9_]*:*').sub('', sent) # 계정 태그(@아이디) -> 삭제
    
    tokens = okt.morphs(okt.normalize(sent)) # 정규화 후 형태소 분석한 토큰
    
    tokens = [token for token in tokens \
             if token not in stop_words and token not in punct\
              and len(token) > 1 and token.isalpha() \
             and 'ㅠ' not in token and 'ㅎ' not in token and 'ㅋ' not in token and 'ㅜ' not in token]
    return tokens

In [13]:
# 년도에 따른 데이터를 넣으면, 해당 속성 단어가 해당 년도에 얼마나 언급되었는지 카운트한 리스트 반환

def get_cnt_by_year(data): #tweets_2019
    data_text = data['Text']
    cnts = []
    
    for proper in properties: # 모든 속성에 대해 검색
        proper_cnt = 0
        for tweet in data_text: # 트윗 데이터의 한 트윗 문장에 대해 반복
            for word in process_sents(tweet): # 전처리한 단어에 대해 반복
                if word in proper: #만약 속성 단어가 포함되어 있다면
                    proper_cnt += 1 # 카운트 더해주기
        cnts.append(proper_cnt)
    
    return cnts 

In [14]:
cnt_2019 = get_cnt_by_year(tweets_2019)
cnt_2020 = get_cnt_by_year(tweets_2020)
cnt_2021 = get_cnt_by_year(tweets_2021)

In [15]:
cnt_2019

[19, 48, 17, 179, 101]

In [16]:
cnt_2020 

[39, 82, 35, 277, 194]

In [17]:
cnt_2021

[51, 76, 32, 291, 122]